In [54]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score



In [7]:

# Define the U-Net architecture
def unet(input_shape):
    inputs = Input(input_shape)

    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=-1)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=outputs)

    return model



In [8]:
# Load and preprocess the images
def load_images(folder_path, target_size):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (target_size[1], target_size[0]))  # Reverse target_size dimensions
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=-1)
        images.append(img)
    return np.array(images)




In [9]:

# Define the input shape
input_shape = (176, 208, 1)

path = r"D:\Research Work Alzheimer\Allfourclassdataset\train"
os.chdir(path)
# Load the images from the folders
mild_demented = load_images('MildDemented', input_shape[:2])
moderate_demented = load_images('ModerateDemented', input_shape[:2])
non_demented = load_images('NonDemented', input_shape[:2])
very_mild_demented = load_images('VeryMildDemented', input_shape[:2])



In [10]:
# Concatenate the images and labels
X = np.concatenate((mild_demented, moderate_demented, non_demented, very_mild_demented), axis=0)
y = np.concatenate((
    np.ones((mild_demented.shape[0],) + input_shape[:2] + (1,)),
    np.ones((moderate_demented.shape[0],) + input_shape[:2] + (1,)) * 2,
    np.ones((non_demented.shape[0],) + input_shape[:2] + (1,)) * 3,
    np.ones((very_mild_demented.shape[0],) + input_shape[:2] + (1,)) * 4,
), axis=0)



In [11]:
# Shuffle the data
random_indices = np.random.permutation(X.shape[0])
X = X[random_indices]
y = y[random_indices]



In [12]:
# Split the data into training and validation sets
split_index = int(0.8 * X.shape[0])
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]



In [13]:
y_val[0]

array([[[3.],
        [3.],
        [3.],
        ...,
        [3.],
        [3.],
        [3.]],

       [[3.],
        [3.],
        [3.],
        ...,
        [3.],
        [3.],
        [3.]],

       [[3.],
        [3.],
        [3.],
        ...,
        [3.],
        [3.],
        [3.]],

       ...,

       [[3.],
        [3.],
        [3.],
        ...,
        [3.],
        [3.],
        [3.]],

       [[3.],
        [3.],
        [3.],
        ...,
        [3.],
        [3.],
        [3.]],

       [[3.],
        [3.],
        [3.],
        ...,
        [3.],
        [3.],
        [3.]]])

In [14]:
# Create and compile the U-Net model
model = unet(input_shape)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


In [15]:
model.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
256/256 [==============================] - 14622s 57s/step - loss: 5.1836 - accuracy: 0.1391 - val_loss: 5.1161 - val_accuracy: 0.1434
Epoch 2/10
256/256 [==============================] - 10450s 41s/step - loss: 5.1699 - accuracy: 0.1392 - val_loss: 5.1161 - val_accuracy: 0.1434
Epoch 3/10
256/256 [==============================] - 8571s 33s/step - loss: 5.1699 - accuracy: 0.1392 - val_loss: 5.1161 - val_accuracy: 0.1434
Epoch 4/10
256/256 [==============================] - 8436s 33s/step - loss: 5.1699 - accuracy: 0.1392 - val_loss: 5.1161 - val_accuracy: 0.1434
Epoch 5/10
256/256 [==============================] - 8430s 33s/step - loss: 5.1699 - accuracy: 0.1392 - val_loss: 5.1161 - val_accuracy: 0.1434
Epoch 6/10
256/256 [==============================] - 17037s 67s/step - loss: 5.1699 - accuracy: 0.1392 - val_loss: 5.1161 - val_accuracy: 0.1434
Epoch 7/10
256/256 [==============================] - 8936s 35s/step - loss: 5.1699 - accuracy: 0.1392 - val_loss: 5.1161 - val

In [16]:
train_predictions = model.predict(X_train)
val_predictions = model.predict(X_val)

33/33 [==============================] - 641s 19s/step


In [17]:
train_predictions.shape

(4096, 176, 208, 1)

In [18]:
train_predictions_flat = train_predictions.reshape(-1, 1)
val_predictions_flat = val_predictions.reshape(-1, 1)

In [19]:
train_predictions_flat.shape

(149946368, 1)

In [20]:
y_train.shape

(4096, 176, 208, 1)

In [25]:
y_train_flat = y_train.reshape(-1)

In [26]:
y_train_flat.shape

(149946368,)

In [27]:
random_forest = RandomForestClassifier()
random_forest.fit(train_predictions_flat, y_train_flat)


RandomForestClassifier()

In [28]:
rf_predictions = random_forest.predict(val_predictions_flat)

In [39]:
rf_predictions.shape

(37523200,)

In [50]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(train_predictions_flat, y_train_flat)

DecisionTreeClassifier()

In [51]:
dt_predictions = decision_tree.predict(val_predictions_flat)

In [52]:
dt_predictions.shape

(37523200,)

In [55]:
knn = KNeighborsClassifier()
knn.fit(train_predictions_flat, y_train_flat)

KNeighborsClassifier()

In [ ]:
knn_predictions = knn.predict(val_predictions_flat)
